# Data generation process
## Overview

This notebook demonstrates the process of generating some synthetic data for causal inference problems.

The diagram below shows how the profit from a customer is related to other factors.
- Profit is the outcome we are interested in.
- Promotion is believed to have a direct impact on the profit. The impact of a promotion also varies based on the average amount of previous orders.
- Profit is also associated with three factors.
  - The income of a customer.
  - The average amount of previous orders.
  - The number of years since registration.

  
Below is a more formal mathematical representation.
- $ y = TE * t + a * x_1 + b * x_2 + c * x_3$
- $ TE = d * x_2^2$


```mermaid
%%{init: {'theme':'default'}}%%

flowchart
subgraph  
direction BT
x1((x1 \n income))
x2((x2 \n avg \n order))
x3((x3 \n yrs since \n registration))
y((y \n profit))
t((t \n promotion))
x1 --a--> y
t --TE--> y
x2 --b--> y
x3 --c--> y
end
```

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

num_samples = 10000
np.random.seed(42)
x1 = abs(np.random.normal(loc=3000, scale=1500, size=num_samples))
x2 = abs(np.random.normal(loc=50, scale=10, size=num_samples))
x3 = abs(np.random.randn(num_samples) + 3) + 0.01
t = np.random.randint(low=0, high=2, size=num_samples)

a = 0.01
b = 0.3
c = 3
d = 2.68

TE = d * x2**2 / 1000
Y = TE * t + a * x1 + b * x2 + c * x3

df = pd.DataFrame({'income': x1, 'avg_order': x2, 'yrs': x3, 'promotion': t, 'TE': TE, 'profit': Y})
df.describe().transpose().drop(columns='count').applymap(lambda x: round(x, 2))

,mean,std,min,25%,50%,75%,max
income,3022.98,1451.88,1.67,1992.69,2996.11,4006.62,8889.36
avg_order,50.14,10.01,11.44,43.38,50.16,56.94,94.79
yrs,3.00,0.99,0.02,2.31,3.00,3.67,6.70
promotion,0.49,0.50,0.00,0.00,0.00,1.00,1.00
TE,7.00,2.72,0.35,5.04,6.74,8.69,24.08
profit,57.73,15.93,15.44,46.43,57.45,68.44,128.91


In [ ]:
print('distribution of promotion')
df['promotion'].value_counts()

In [ ]:
df.head()

In [ ]:
px.box(df, x='profit', color='promotion', width=800, height=300, title='The distribution of profit split by promotion')

## Treatment effect estimation

In many scenarios, we want to find out the uplift/increase in profit from a customer that is driven by promotions. It is equivalent to estimating the treatment effect of running a promotion on a customer. 

Since the treatment effect varies based on another attribute, it means that the treatment effect is heterogeneous and we want to model conditional average treatment effects (CATE). Being able to do that would enable us to identify the groups of customers who respond positively to promotions, which means that we would be able to run more sophisticated campaigns to maximise the return.

The more professional term describing the process above is **uplift modelling**. There are several algorithms we can use. I will start with talking about meta learner.

### Meta learner

Meta learner is a technique that integrates information from one or multiple machine learning models to generate combined estimate of causal effects. It includes S-learner, T-learner, X-learner. 
We will be utilising the implementation of EconML.

#### S-learner​
- Train only one model M() using both treated and control samples.
- Takes both the treatment T and features X to predict the outcome.​
- Scoring: the treatment effect is M(X|T=1) - M(X|T=0)


```mermaid
%%{init: {'theme':'default'}}%%
flowchart
subgraph S_learners
direction LR
subgraph training
direction TB
subgraph all_samples
direction TB
X
t
y
end
input_train[input]
input_train-->m0
target_train[target]
X-->input_train
t-->input_train
y-->target_train
target_train-->m0
m0[(model M)]
end

subgraph predicting
direction TB
input[X]
model[(model M)]
t0[X, t=0]
t1[X, t=1]
t0-->model
t1-->model
input-->t0
input-->t1
y0[y0]
y1[y1]
model --predicts-->y0
model --predicts-->y1
cate[treatment effect/uplift = y1 - y0]

y0-->cate
y1-->cate
end
training --> predicting
end
```

In [ ]:
from econml.metalearners import SLearner
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

t = df['promotion'].to_numpy()
X = df[['income', 'avg_order', 'yrs']].to_numpy()
y = df['profit'].to_numpy()
TE = df['TE'].to_numpy()

X_train, X_test, y_train, y_test, t_train, t_test, TE_train, TE_test = train_test_split(X, y, t, TE, random_state=42)

In [ ]:
for matrix, name in zip([X_train, X_test, y_train, y_test, t_train, t_test, TE_train, TE_test], ['X_train', 'X_test', 'y_train', 'y_test', 't_train', 't_test', 'TE_train', 'TE_test']):
    print(name+' shape: ', matrix.shape)

In [ ]:
df_test = pd.DataFrame()

In [ ]:
slearner = SLearner(overall_model=RandomForestRegressor(random_state=42))
slearner.fit(Y=y_train, T=t_train, X=X_train)

In [ ]:
TE_pred = slearner.effect(X_test)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(TE_test, TE_pred))
rmse

In [ ]:
px.scatter(x=TE_test, y=TE_pred, width=500, height=500, 
           title=f'Correlation score: {round(np.corrcoef(TE_test, TE_pred)[0][1], 2)}',
           labels={'x':'TE_test', 'y': 'TE_pred'},
           trendline='ols')

In [ ]:
px.box(x=(TE_pred - TE_test)/TE_test, width=600, height=400, title='(TE_pred-TE_true)/TE_true')

#### T-learner
- Handle regularisation bias issue from S-learner​
- Train two models F_1() and F_0(). F_1 uses treated samples, F_0 uses control samples.​
- Both model take only features X to predict the outcome Y.​
- Scoring: the treatment effect is F_1(X) - F_0(T=0)​



```mermaid
%%{init: {'theme':'default'}}%%
flowchart
subgraph T_learners
subgraph training
subgraph all_samples
X
t
y
end
subgraph control_samples,t=0
x_control[X]
y_control[y]
end

subgraph treated_samples,t=1
x_treatment[X]
y_treatment[y]

end
all_samples --> control_samples,t=0
all_samples --> treated_samples,t=1
m0[(control model M_0)]
m1[(treatment model M_1)]
input_train_control-->m0
input_train_treatment-->m1
target_train_control-->m0
target_train_treatment-->m1
input_train_control[input]
input_train_treatment[input]
target_train_control[target]
target_train_treatment[target]

x_control-->input_train_control
y_control-->target_train_control
x_treatment-->input_train_treatment
y_treatment-->target_train_treatment

end

subgraph predicting
input[X]
m0_pred[(control model M_0)]
m1_pred[(treatment model M_1)]
input --> m0_pred
input --> m1_pred
y0[y0]
y1[y1]
m0_pred --predicts-->y0
m1_pred --predicts-->y1
cate[treatment effect/uplift = y1 - y0]

y0-->cate
y1-->cate
end
training --> predicting
end
``````


In [ ]:
from econml.metalearners import TLearner

In [ ]:
tlearner = TLearner(models=RandomForestRegressor(random_state=42))
tlearner.fit(Y=y_train, T=t_train, X=X_train)

In [ ]:
TE_pred = tlearner.effect(X_test)
rmse = np.sqrt(mean_squared_error(TE_test, TE_pred))
rmse

In [ ]:
px.scatter(x=TE_test, y=TE_pred, width=500, height=500, 
           title=f'Correlation score: {round(np.corrcoef(TE_test, TE_pred)[0][1], 2)}',
           labels={'x':'TE_test', 'y': 'TE_pred'},
           trendline='ols')

In [ ]:
px.box(x=(TE_pred - TE_test)/TE_test, width=600, height=400, title='(TE_pred-TE_true)/TE_true')

### DML

The next algorithm is DML, a two-stage framework to estimate causal parameters, offers great flexibility to choose first and second stage models.​ It is good at handling regularisation bias and confounding bias​.

- Stage 1: Partialling-out phase (remove the effects from confounders in treatment and outcome)
- Stage 2: Heterogeneity modelling phase (regress y_res using t_res and features)​

In this exercise, I will be using one of the variants called CausalForsetDML implemented in EconML.


```mermaid
%%{init: {'theme':'default'}}%%
flowchart
subgraph stage_1
direction TB
subgraph  
mt[(treatment model, M_t)]
my[(outcome model, M_y)]
t_pred[treatment predicted, t_pred]
y_pred[outcome predicted, y_pred]
t_res[treatment residual, t_res]
y_res[outcome residual, y_res]
mt-->t_pred
my-->y_pred
t_pred--t_true - t_pred-->t_res
y_pred--y_true - y_pred-->y_res
input[input]
target_y[target]
target_t[target]
target_y-->my
target_t-->mt
input-->mt
input-->my
end
feature[feature, x]
treatment --> target_t
feature --> input
treatment[treatment, t]
outcome[outcome, y]
outcome --> target_y
end
```

```mermaid
%%{init: {'theme':'default'}}%%
flowchart
subgraph  
direction TB
subgraph stage_2
direction TB
t_res
y_res[y_res]
target
y_res-->target
t_res[t_res]
t_res-->input
input
feature[feature, x]
feature-->input
model[(CATE model, M_cate)]
input-->model
target-->model
cate[treatment effect/uplift for each sample]
model-- output -->cate
end
end
```


In [ ]:
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestClassifier

cfdml = CausalForestDML(model_y=RandomForestRegressor(random_state=42),
                        model_t=RandomForestClassifier(random_state=42),
                        discrete_treatment=True)

cfdml.fit(Y=y_train, T=t_train, X=X_train)

In [ ]:
TE_pred = cfdml.effect(X_test)
rmse = np.sqrt(mean_squared_error(TE_test, TE_pred))
rmse

In [ ]:
px.scatter(x=TE_test, y=TE_pred, width=500, height=500, 
           title=f'Correlation score: {round(np.corrcoef(TE_test, TE_pred)[0][1], 2)}',
           labels={'x':'TE_test', 'y': 'TE_pred'},
           trendline='ols')

In [ ]:
px.box(x=(TE_pred - TE_test)/TE_test, width=600, height=400, title='(TE_pred-TE_true)/TE_true')

### Comparison